In [1]:
import pandas as pd
import numpy as np

# Data Loading

In [2]:
datos_clima =  pd.read_pickle("etlData/datos_clima.pickle")
datos_clima

,Temp_Air_Mean,Temp_Air_Min,Temp_Air_Max,Dew_Temp_Mean,Relat_Hum_Mean,Relat_Hum_Min,Relat_Hum_Max,Wind_Speed_Mean,Wind_Speed_Min,Wind_Speed_Max
Date,,,,,,,,,,
2017-01-05 19:00:00-05:00,24.00,21.39,29.39,21.28,85.0,64.0,96.0,1.03,0.00,3.08
2017-01-06 19:00:00-05:00,22.67,21.11,27.00,20.00,85.0,67.0,92.0,2.32,0.09,5.72
2017-01-07 19:00:00-05:00,22.56,19.78,26.61,20.00,86.0,67.0,96.0,1.03,0.00,5.90
2017-01-08 19:00:00-05:00,22.44,20.22,25.89,19.39,83.0,70.0,92.0,1.70,0.00,5.72
2017-01-09 19:00:00-05:00,23.89,20.72,31.78,19.22,76.0,48.0,90.0,1.97,0.00,5.72
...,...,...,...,...,...,...,...,...,...,...
2023-11-25 00:00:00-05:00,26.00,21.00,32.00,21.00,47.0,91.0,51.0,2.00,1.00,3.00
2023-11-26 00:00:00-05:00,25.00,22.00,28.00,20.00,65.0,84.0,65.0,2.00,2.00,3.00
2023-11-27 00:00:00-05:00,25.00,22.00,30.00,20.00,57.0,85.0,57.0,2.00,1.00,3.00


In [3]:
tendency_exclude = ["Wind_Speed_Min"]

# Prep before training

In [4]:
future_year = 2025


# Model Training

## SARIMAX

In [5]:
!pip install pmdarima

In [6]:
from pmdarima.arima import auto_arima
import timeseries as ts

In [7]:
from importlib import reload  # Python 3.4+
ts = reload(ts)

In [8]:
import pickle as pkl

with open("./variablesSeleccionadas.pkl", "rb+") as f:
    variables_seleccionadas = pkl.loads(f.read())
variables_seleccionadas

{'Tommy Atkins': array(['Wind_Speed_Mean', 'Relat_Hum_Mean', 'Wind_Speed_Min',
        'Relat_Hum_Max'], dtype=object),
 'Ataulfo': array(['Wind_Speed_Mean', 'Relat_Hum_Mean'], dtype=object)}

In [9]:
from functools import reduce

In [10]:
variables_seleccionadas = reduce(lambda s1,s2: s1 | s2, 
    list(set(valores) for valores in variables_seleccionadas.values()), 
    set()
)

variables_seleccionadas

{'Relat_Hum_Max', 'Relat_Hum_Mean', 'Wind_Speed_Mean', 'Wind_Speed_Min'}

In [11]:
from datetime import datetime
from collections import defaultdict

In [12]:
cutoff = datetime(2022,6,2)
arimas = dict()
all_train = dict()
all_test = dict()

In [13]:
for variable in variables_seleccionadas:
    data = datos_clima[variable]
    train,test = ts.train_test_split(data, cutoff)
    all_train[variable] = train
    all_test[variable] = test
#arimas

/Notebooks/timeseries.py:5: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  train = timeseries.loc[:cutoff]
/Notebooks/timeseries.py:6: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.
  test = timeseries.loc[cutoff:]


In [21]:
for variable in variables_seleccionadas:
    data = datos_clima[variable].resample("M").mean()
    train = all_train[variable]
    autoARIMA = auto_arima(train, seasonal=True, m=12)
    autoARIMA.fit(train)
    arimas[variable] = autoARIMA
    print(f"Trained ARIMA for {variable} :: {autoARIMA.summary()}")
#arimas

Trained ARIMA for Wind_Speed_Mean ::                                SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                 1745
Model:               SARIMAX(2, 1, 1)   Log Likelihood                -976.169
Date:                Tue, 23 Jul 2024   AIC                           1960.338
Time:                        03:51:08   BIC                           1982.193
Sample:                             0   HQIC                          1968.418
                               - 1745                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.4153      0.021     19.534      0.000       0.374       0.457
ar.L2          0.1137      0.023      4.912      0.000       0.068       0.159
ma.L1         -

In [15]:
import sarimax_utils as srm

In [16]:
all_predictions = dict()
metrics = pd.DataFrame(columns=["Variable", "RMSE"])
for variable in variables_seleccionadas:
    autoARIMA = arimas[variable]
    predictions = srm.predict(autoARIMA, 12)
    all_predictions[variable] = predictions
all_predictions

/home/jupyter/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/jupyter/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/jupyter/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/jupyter/anaconda3/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


{'Wind_Speed_Mean':            min     value       max
 1745 -0.028999  0.800699  1.630398
 1746 -0.101302  0.810836  1.722974
 1747 -0.090792  0.865167  1.821126
 1748 -0.087024  0.888882  1.864788
 1749 -0.082223  0.904909  1.892041
 1750 -0.079942  0.914262  1.908465
 1751 -0.079294  0.919968  1.919231
 1752 -0.079866  0.923402  1.926670
 1753 -0.081219  0.925477  1.932172
 1754 -0.083065  0.926729  1.936523
 1755 -0.085215  0.927485  1.940184
 1756 -0.087548  0.927941  1.943430,
 'Wind_Speed_Min':            min          value       max
 1745 -0.097652  4.592750e-239  0.097652
 1746 -0.110058   0.000000e+00  0.110058
 1747 -0.110058   0.000000e+00  0.110058
 1748 -0.110058   0.000000e+00  0.110058
 1749 -0.110058   0.000000e+00  0.110058
 1750 -0.110058   0.000000e+00  0.110058
 1751 -0.110058   0.000000e+00  0.110058
 1752 -0.110058   0.000000e+00  0.110058
 1753 -0.110058   0.000000e+00  0.110058
 1754 -0.110058   0.000000e+00  0.110058
 1755 -0.110058   0.000000e+00  0.110058
 1

In [17]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [18]:
metrics = pd.DataFrame(columns=["Variable"])
for variable in all_predictions:
    autoARIMA = arimas[variable]
    test = all_test[variable]
    predictions = all_predictions[variable]
    row = {
        "Variable": variable, 
        "RMSE": np.sqrt(mean_squared_error(
            test.iloc[0:12], 
            predictions.iloc[0:12]["value"], 
        )),
        "MAE": mean_absolute_error(
            test.iloc[0:12], 
            predictions.iloc[0:12]["value"], 
        ),
        "MAPE": mean_absolute_percentage_error(
            test.iloc[0:12], 
            predictions.iloc[0:12]["value"], 
        ),
    }
    metrics = metrics.append(
        row,
        ignore_index=True,
    )
metrics.sort_values("RMSE", ascending=False)

/tmp/ipykernel_658/221005650.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append(
/tmp/ipykernel_658/221005650.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append(
/tmp/ipykernel_658/221005650.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append(
/tmp/ipykernel_658/221005650.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics = metrics.append(


,Variable,RMSE,MAE,MAPE
3,Relat_Hum_Mean,8.233194,6.732500e+00,9.353612e-02
2,Relat_Hum_Max,3.902830,2.417091e+00,2.655632e-02
0,Wind_Speed_Mean,0.400292,3.512838e-01,3.784971e-01
1,Wind_Speed_Min,0.000000,3.827292e-240,1.723659e-224


In [19]:
type(arimas["Relat_Hum_Max"])

pmdarima.arima.arima.ARIMA

In [20]:
import os
MODELS_DIR="models"
os.makedirs(MODELS_DIR, exist_ok=True)
ARIMAS_PATH = f"{MODELS_DIR}/arimas.pkl"

with open(ARIMAS_PATH, "wb") as f:
    pkl.dump(arimas, f)